# stocks 정보 최신화

In [ ]:
# pykrx to mysql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from openpyxl import load_workbook
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'test'

ticker_list = []

try:  
  for ticker in stock.get_market_ticker_list(market='KOSPI'):
    ticker_list.append(ticker)
  time.sleep(1)
  for ticker in stock.get_market_ticker_list(market='KOSDAQ'):
    ticker_list.append(ticker)

  start = "20211001"
  end = str(datetime.now().date()).replace('-', '')

  for ticker in ticker_list:
    try:
      name = stock.get_market_ticker_name(ticker)
      time.sleep(1)
      df = stock.get_market_ohlcv_by_date(start, end, ticker).drop(['시가', '고가', '저가'], axis=1).rename(columns={'종가':'close', '거래량':'volume'})
      time.sleep(1)
      df.index.names = ['date']
      df_volume_netbuy = stock.get_market_trading_volume_by_date(start, end, ticker, on="순매수").rename(columns={'기관합계':'insVol', '기타법인':'etcVol', '개인':'indVol', '외국인합계':'forVol'}).drop(['전체'], axis=1)
      time.sleep(1)
      df_amount_netbuy = stock.get_market_trading_value_by_date(start, end, ticker, on="순매수").rename(columns={'기관합계':'insAmt', '기타법인':'etcAmt', '개인':'indAmt', '외국인합계':'forAmt'}).drop(['전체'], axis=1)
      time.sleep(1)
      df_total = pd.concat([df, df_volume_netbuy, df_amount_netbuy], axis = 1, join='inner')
      df_total.insert(0, 'ticker', str(ticker))
      df_total.insert(1, 'name', name)

      db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
      conn = db_connection.connect()

      df_total.to_sql(name='stocks', con=db_connection, if_exists='append', index=True, index_label="date")

      conn.close()
    except Exception as e:
      print('e:', ticker, e)
      pass
except Exception as ex:
  print('ex: ', ticker, ex)
  pass

# 볼린저밴드 구하기

In [43]:
import pandas as pd
import pymysql
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  for ticker in tickers:        
    try:
      getLast20Info = "SELECT * FROM stocks WHERE ticker = '" + ticker[0] + "' ORDER BY date DESC limit 20"
      cursor.execute(getLast20Info)
      last20InfoDf = pd.DataFrame(cursor.fetchall())
      last20InfoDfClose = last20InfoDf.loc[:, 3]

      avg = last20InfoDfClose.mean()
      std = last20InfoDfClose.std()

      tmpInfo = last20InfoDf.loc[0, [0, 1, 2, 3]]
      hi = round(avg + std * 2)
      me = round(avg)
      lo = round(avg - std * 2)
      insertBollSql = "REPLACE INTO boll(date, ticker, name, close, low, medium, high) VALUES(%s, %s, %s, %s, %s, %s, %s)"
      cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], tmpInfo[2], tmpInfo[3], lo, me, hi))
      db.commit()
    except Exception as ex1:
      print('ex1', ex1)
      # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], tmpInfo[3], hi, me, lo)
      pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

ex1 cannot convert float NaN to integer
ex1 cannot convert float NaN to integer
ex1 cannot convert float NaN to integer


In [39]:
last20InfoDf.loc[:, [0, 1, 2, 3]]
# last20InfoDf
last20InfoDfClose = last20InfoDf.loc[:, 3]

avg = last20InfoDfClose.mean()
std = last20InfoDfClose.std()

# print(type(avg))
hi = round(avg + std * 2)
mi = round(avg)
lo = round(avg - std * 2)
print(hi, mi, lo)
last20InfoDf.loc[0, [0, 1, 2, 3]]

46623 43578 40532


0    2021-09-30
1        240810
2         원익IPS
3         40050
Name: 0, dtype: object